In [1]:
import networkx as nx
import numpy as np
from numpy.random import choice
from collections import OrderedDict
import json

In [2]:
seed = 17
np.random.seed(17)

np.set_printoptions(precision=2)

cluster_r_size = 50
cluster_b_size = 50
wide_bridge_size = 3

number_of_cascades = 100

In [3]:
#Generate the base graph with two clusters the wide bridge and no attachments

sg1 = nx.powerlaw_cluster_graph(cluster_r_size, 3, 0.05, seed=seed)
nx.set_node_attributes(sg1, 'r', 'color')

sg2 = nx.powerlaw_cluster_graph(cluster_b_size, 3, 0.05, seed=seed)
nx.set_node_attributes(sg2, 'b', 'color')
sg2 = nx.relabel_nodes(sg2, {i:i + cluster_r_size for i in range(cluster_b_size)})

sgWB = nx.Graph()
sgWB.add_nodes_from(range(cluster_r_size+cluster_b_size,cluster_r_size+cluster_b_size + wide_bridge_size))
nx.set_node_attributes(sgWB, 'g', 'color')

sg = nx.compose(sg1, sg2)
sg = nx.compose(sg, sgWB)
colors_1 = [sg.nodes[n]['color'] for n in sg.nodes()]

# Generate a basic attachment
sg_1 = sg.copy()
cluster_r_size = 50
cluster_b_size = 50
wide_bridge_size = 3
for i in range(cluster_b_size+cluster_r_size,cluster_b_size+cluster_r_size+wide_bridge_size):
    sg_1.add_edge(np.random.randint(cluster_r_size),i)
    sg_1.add_edge(cluster_r_size+np.random.randint(cluster_b_size),i)

In [4]:
# Generate a complex attachment

# Use an geometric distribution to determine how many targets in each cluster to attach to
# Use a preferential attachment distribution over the cluster to determine which get attachments
# Have wide bridge connectors connect to eachother with bernoulli dist

def generate_graph_with_attachments(attachment_number_mean = 1, inter_bridge_connection_prob =  0.05, cluster_r_size = 50,cluster_b_size = 50, wide_bridge_size = 3):
    sg1 = nx.powerlaw_cluster_graph(cluster_r_size, 3, 0.05, seed=seed)
    nx.set_node_attributes(sg1, 'r', 'color')

    sg2 = nx.powerlaw_cluster_graph(cluster_b_size, 3, 0.05, seed=seed)
    nx.set_node_attributes(sg2, 'b', 'color')
    sg2 = nx.relabel_nodes(sg2, {i:i + cluster_r_size for i in range(cluster_b_size)})

    sgWB = nx.Graph()
    sgWB.add_nodes_from(range(cluster_r_size+cluster_b_size,cluster_r_size+cluster_b_size + wide_bridge_size))
    nx.set_node_attributes(sgWB, 'g', 'color')

    sg = nx.compose(sg1, sg2)
    sg = nx.compose(sg, sgWB)
    colors = [sg.nodes[n]['color'] for n in sg.nodes()]
    
    attachment_numbers_r = np.random.geometric(1/attachment_number_mean, size=wide_bridge_size)
    attachment_numbers_b = np.random.geometric(1/attachment_number_mean, size=wide_bridge_size)

    pa_r = [x[1] for x in sg1.degree]/np.sum([x[1] for x in sg1.degree])
    pa_b = [x[1] for x in sg2.degree]/np.sum([x[1] for x in sg2.degree])

    def to_boolean_array(size, indices):
        res = np.zeros(size)
        res[indices] = 1
        return res

    attachments_r =  np.array([to_boolean_array(cluster_r_size,choice(cluster_r_size, x, p=pa_r)) for x in attachment_numbers_r])
    attachments_b =  np.array([to_boolean_array(cluster_b_size,choice(cluster_b_size, x, p=pa_b)) for x in attachment_numbers_b])

    inter_bridge_attachments = np.random.binomial(1, inter_bridge_connection_prob, size=(wide_bridge_size,wide_bridge_size))
    
    new_edges = [(x,cluster_r_size+cluster_b_size+y) for x in range(cluster_r_size) for y in range(wide_bridge_size) if attachments_r.T[x,y]] 
    new_edges += [(cluster_r_size+x,cluster_r_size+cluster_b_size+y) for x in range(cluster_b_size) for y in range(wide_bridge_size) if attachments_b.T[x,y]]
    new_edges += [(cluster_r_size+cluster_b_size+x,cluster_r_size+cluster_b_size+y) for x in range(wide_bridge_size) for y in range(wide_bridge_size) if inter_bridge_attachments.T[x,y]]

    sg_res = sg.copy()
    sg_res.add_edges_from(new_edges)
    return sg_res, colors


sg_2, colors_2 = generate_graph_with_attachments(2, 0.05, 10, 10, 3)
sg_3, colors_3 = generate_graph_with_attachments(2, 0.05, 50, 50, 3)
sg_4, colors_4 = generate_graph_with_attachments(2, 0.05, 100, 100, 3)
sg_5, colors_5 = generate_graph_with_attachments(1, 0.05, 50, 50, 40)
sg_6, colors_6 = generate_graph_with_attachments(5, 0.1)

graphs_and_colors = [(1, sg_1, colors_1),(2, sg_2, colors_2),(3, sg_3, colors_3),(4, sg_4, colors_4),(5, sg_5, colors_5),(6, sg_6, colors_6)]

In [5]:
for i, graph, colors in graphs_and_colors:
    with open('graph_'+str(i)+'.txt','wb') as f:
        for line in nx.to_numpy_matrix(graph):
            np.savetxt(f, line, fmt='%.2f')
    with open('colors_'+str(i)+'.txt', 'w') as f:
        for color in colors:
            f.write(color)

In [6]:
def generate_cascade(graph_t, source):
    graph = graph_t.copy()
    graph_size = len(graph.nodes)
    r = np.random.weibull(2, (graph_size, graph_size))
    s = np.random.uniform(size=(graph_size, graph_size))
    tau = -1*np.log(s)/r
    tau = np.multiply((nx.to_numpy_matrix(graph)), tau)

    graph.add_weighted_edges_from([(x,y,tau[x,y]) for x in range(graph_size) for y in range(graph_size) if tau[x,y] > 0])
    
    source = source
    branching = nx.shortest_path(graph, source, weight='weight')
    timings = nx.shortest_path_length(graph, source, weight='weight')
    
    return (branching, timings)

def generate_path_from_prev(prev_dict, mem, u):
    if u in mem:
        return mem[u], mem
    else:
        if prev_dict[u] is not None:
            new_path, new_mem = generate_path_from_prev(prev_dict, mem, prev_dict[u])
            mem[u] = new_path + [u]
            return mem[u], mem
        else:
            mem[u] = [u]
            return [u], mem
    
def generate_paths(prev_dict, source):
    #returns paths for u, with updated dict
    mem = {}
    for u in prev_dict.keys():
        generate_path_from_prev(prev_dict, mem, u)
    for k in prev_dict.keys():
        if prev_dict[k] is None and k != source:
            del mem[k]
    return mem

def shortest_path(graph, source):
    q = OrderedDict()
    prev = {}
    timings = {}
    for v in graph.nodes():
        q[v] = float('inf')
        timings[v] = None
        prev[v] = None
    q[source] = 0
    timings[source] = 0
    while q != {}:
        q = OrderedDict(sorted(q.items(), key=lambda x: x[1], reverse=True))
        u, d = q.popitem()
        for v in graph.neighbors(u):
            if v in q.keys():
                alt = d + graph[u][v]['weight']
                if alt < q[v]:
                    q[v] = alt
                    prev[v] = u
                    timings[v] = alt
    return timings, prev

def complex_shortest_path(graph, source, complex_criteria, sus):
    q = OrderedDict() #Essentially a queue of infected waiting to infect
    prev = {}
    timings = {} # Time till first infection
    infected = set()
    for v in graph.nodes():
        timings[v] = float('inf')
        prev[v] = None
    q[source] = 0 # Enqueue the start note //
    infected.add(source) #Infect the source
    timings[source] = 0 
    while q != {}:
        q = OrderedDict(sorted(q.items(), key=lambda x: x[1], reverse=True))
        u, d = q.popitem() #Dequeue an infected node, with respect to how quickly it was infected
        for v in graph.neighbors(u):
            if v not in infected:
                alt = d + graph[u][v]['weight']
                criteria, sus = complex_criteria(v, infected, sus)
                if criteria:
                    q[v] = alt
                    timings[v] = alt
                    prev[v] = u
                    infected.add(v)
    return timings, prev

# Rohit's original hard criteria version
def complex_generate_cascade_with_hard(graph_t, source, limit=3):
    graph = graph_t.copy()
    graph_size = len(graph.nodes)
    r = np.random.weibull(2, (graph_size, graph_size))
    s = np.random.uniform(size=(graph_size, graph_size))
    tau = -1*np.log(s)/r
    tau = np.multiply((nx.to_numpy_matrix(graph)), tau)
    
    sus = np.random.uniform(0,limit, graph_size)
    
    graph.add_weighted_edges_from([(x,y,tau[x,y]) for x in range(graph_size) for y in range(graph_size) if tau[x,y] > 0])
    
    def complex_criteria(node,infected, sus=sus, graph = graph):
        return sum([1 for n in graph.neighbors(node) if n in infected]) > sus[node], sus
    
    source = source
    timings, prev = complex_shortest_path(graph, source, complex_criteria, sus)
    branching = generate_paths(prev, source)
    for k in prev.keys():
        if timings[k] == float('inf'):
            del timings[k]
    
    return (branching, timings)

# Andrei's version of complex contagion
def complex_generate_cascade_with_prob(graph_t, source, gamma=0.1):
    graph = graph_t.copy()
    graph_size = len(graph.nodes)
    r = np.random.weibull(2, (graph_size, graph_size))
    s = np.random.uniform(size=(graph_size, graph_size))
    tau = -1*np.log(s)/r
    tau = np.multiply((nx.to_numpy_matrix(graph)), tau)
    
    sus = np.random.uniform(0,1, graph_size)
    
    graph.add_weighted_edges_from([(x,y,tau[x,y]) for x in range(graph_size) for y in range(graph_size) if tau[x,y] > 0])
    
    def complex_criteria(node,infected, sus=sus, graph = graph, gamma=gamma):
        sus[node] += gamma
        sus[node] = min(1, sus[node])
        if node in infected:
            return True, sus
        chance = np.random.binomial(1,sus[node])
        return chance == 1, sus
    
    source = source
    timings, prev = complex_shortest_path(graph, source, complex_criteria, sus)
    branching = generate_paths(prev, source)
    for k in prev.keys():
        if timings[k] == float('inf'):
            del timings[k]
    
    return (branching, timings)

# Uses the fraction of the friends infected as the criteria
def complex_generate_cascade_with_ltm(graph_t, source):
    graph = graph_t.copy()
    graph_size = len(graph.nodes)
    r = np.random.weibull(2, (graph_size, graph_size))
    s = np.random.uniform(size=(graph_size, graph_size))
    tau = -1*np.log(s)/r
    tau = np.multiply((nx.to_numpy_matrix(graph)), tau)
    
    sus = np.random.uniform(0,1, graph_size)
    
    graph.add_weighted_edges_from([(x,y,tau[x,y]) for x in range(graph_size) for y in range(graph_size) if tau[x,y] > 0])
    
    def complex_criteria(node,infected, sus=sus, graph = graph):
        return np.mean([(1 if n in infected else 0) for n in graph.neighbors(node)]) > sus[node], sus
    
    source = source
    timings, prev = complex_shortest_path(graph, source, complex_criteria, sus)
    branching = generate_paths(prev, source)
    for k in prev.keys():
        if timings[k] == float('inf'):
            del timings[k]
    
    return (branching, timings)

def does_cross_bridge(cascade, colors):
    argmin = (lambda d: min(d, key=d.get))
    source = argmin(cascade[1])
    if colors[source] == 'g':
        return False
    not_source_color = 'r' if colors[source] == 'b' else 'b'
    return any([colors[node] == not_source_color  for node in cascade[1].keys()])
            
def generate_single_cascade(graph, colors, complex_method='Prob', does_cross=True, single_source=None, params={'limit':3, 'gamma':0.1}):
    # The options for the complex contagion method are None, 'Hard', 'Prob' and 'LTM'
    MAX_ITER = 1000
    counter = 0
    while counter < MAX_ITER:
        if single_source is None:
            graph_size = len(graph.nodes)
            source = np.random.randint(graph_size-1)
        else:
            source = single_source
        counter += 1
        if complex_method is None:
            gen_cas = generate_cascade
        elif complex_method == 'Hard' and 'limit' in params:
            gen_cas = complex_generate_cascade_with_hard
            args = {'limit' : params['limit']}
        elif complex_method == 'Prob' and 'gamma' in params:
            gen_cas = complex_generate_cascade_with_prob
            args = {'gamma' : params['gamma']}
        elif complex_method == 'LTM':
            gen_cas = complex_generate_cascade_with_ltm
            args = {}
        else:
            raise Exception('Not a valid contagion method and/or parameter')
        cas = gen_cas(graph, source, **args)
        if not does_cross:
            return cas
        if does_cross and does_cross_bridge(cas, colors):
            return cas
    logging.warning('Max Iterations exceeded when generating cascades')

In [7]:
cascades = [generate_single_cascade(graph, colors) for _ in range(number_of_cascades)]

In [8]:
file = open('example_cascades.json', 'w')
json.dump([{'branching': cas[0], 'timing': cas[1]} for cas in cascades], file)